**Summary:** 
This tutorial shows the basic functions and usages of btk. For a more detailed understanding of the btk pipeline refer to the [User Guide](https://blendingtoolkit.readthedocs.io/en/latest/user_guide.html) on the package [website](https://blendingtoolkit.readthedocs.io/en/latest/index.html). 

For instructions on how to download required dependencies take a look at this [page](https://blendingtoolkit.readthedocs.io/en/latest/install.html).

# Setup

In [ ]:
# Autoreload packages in case they change.
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

In [ ]:
sys.path.insert(0,os.path.dirname(os.getcwd())) # add BlendingToolKit so we can import btk 

In [ ]:
import btk
import btk.plot_utils
import btk.survey
import btk.draw_blends
import astropy.table

# Draw some blends

In [ ]:
from btk.survey import Rubin

In [ ]:
stamp_size = 24.0 # arcsecs
catalog_name = "../data/sample_input_catalog.fits"
catalog = btk.catalog.CatsimCatalog.from_file(catalog_name)
sampling_function = btk.sampling_functions.DefaultSampling(stamp_size=stamp_size, max_number=3)
draw_generator = btk.draw_blends.CatsimGenerator(
    catalog,
    sampling_function,
    [Rubin],
    batch_size=8,
    stamp_size=stamp_size,
    shifts=None,
    indexes=None,
    multiprocessing=False,
    cpus=1,
    add_noise=True,
    snr=200
)

In [ ]:
Rubin.pixel_scale

In [ ]:
batch = next(draw_generator)
blend_images = batch['blend_images']
blend_list = batch['blend_list']
btk.plot_utils.plot_blends(blend_images, blend_list, limits=(30,90))

# Basic Measurement with SEP

In [ ]:
#this is how we specify the measurement params
meas_params = btk.measure.BasicMeasureParams()

In [ ]:
# get generator that yields measured values by running the algorithm in the images from the draw_blend_generator. 
# you can also turn on multiprocessing in this step in case your algorithm is CPU intensive. 
meas_generator = btk.measure.MeasureGenerator(meas_params, draw_generator)

In [ ]:
output, deb, _ = next(meas_generator)  # obtain simulation measurement output for this next batch. 
blend_list = output['blend_list']
for i in range(len(blend_list)):
    blend_images = output['blend_images'][i:i+1]
    detected_centers = deb[i]['peaks']
    true_centers = np.stack([blend_list[i]['dx'], blend_list[i]['dy']]).T
    true_table = blend_list[i]
    det_table = astropy.table.Table(detected_centers, names=('dx', 'dy'))
    _, _, summary = btk.metrics.evaluate_detection([true_table], [det_table], 0)
    det, undet, spur = summary[0][1:4]
    print("Out of %i objects, %i detected, %i undetected, & %i  spurious detections"%(len(true_centers), det, undet, spur))
    btk.plot_utils.plot_blends(blend_images, blend_list[i: i+1], [detected_centers],limits=(30,90))